In [ ]:
import pandas as pd
import numpy as np
import common_libs.utilities as ut

In [ ]:
cnx = ut.create_connection()
t = pd.read_sql('SELECT * FROM time JOIN time_metadata ON time.time_id=time_metadata.time_id', cnx).set_index('time_id')

In [ ]:
good_times = t[(t['l1drmisses'] <= 0) & (t['l1dwmisses'] <= 0) & (t['l1imisses'] <= 0) & (t['conswitch'] <= 0)]

In [ ]:
def percentile(k):
    return lambda a: np.percentile(a, k)

In [ ]:
summaries = good_times.groupby('code_id').agg({'code_id': 'count', 'cycle_count': percentile(20)})

In [ ]:
true_cycles = summaries[summaries['code_id'] > 1]['cycle_count']

In [ ]:
t_intel = pd.read_sql('SELECT * FROM time WHERE kind_id=2', ut.create_connection()).groupby('code_id').mean()['cycle_count']
err = ((true_cycles - t_intel).abs() / true_cycles).dropna()
print('Intel: {} blocks\nAverage error {}'.format(
    len(err),
    err.mean(),
))

In [ ]:
t_llvm = pd.read_sql('SELECT * FROM time WHERE kind_id=3', ut.create_connection()).groupby('code_id').mean()['cycle_count']
err = ((true_cycles - t_llvm).abs() / true_cycles).dropna()
print('LLVM: {} blocks\nAverage error {}'.format(
    len(err),
    err.mean(),
))